# Anti-Spoof Face Recognition System (FOR GITHUB)

In [1]:
import pickle
import cv2
from imutils.video import VideoStream, FileVideoStream
import imutils
from imutils import paths
import face_recognition as fr
from imutils import face_utils
import dlib
from scipy.spatial import distance as dist
import numpy as np
import time

from collections import Counter

### Function to calculate eye-aspect ratio | Initiate variables | retrieve start and end indexes of  left and right eye

In [2]:
def eye_aspect_ratio(eye): 
    v1 = dist.euclidean(eye[1],eye[5])
    v2  = dist.euclidean(eye[2],eye[4]) 
    h = dist.euclidean(eye[0],eye[3])
    
    return (v1+v2)/(2*h) 

#Initialize variables
EAR_thresh = 0.3

BLINK_CONSEC_FRAMES = 3

blink_threshold = 5

count = 0
blinks = 0 

#Initialize the Face Detector and Landmark Detector
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")


from imutils.face_utils import FACIAL_LANDMARKS_IDXS as landmark_inds

(lstart,lend) = landmark_inds['left_eye'] 
(rstart,rend) = landmark_inds['right_eye'] 

### Function to convert rectangle object into coordinates-format that 'face_encodings' accepts.

In [3]:
def rect_to_list(rect):
        x = rect.left()
        y = rect.top()
        r = rect.right()
        b = rect.bottom()
        return (y,r,b,x) 

In [4]:
f = open('encodings.pickle', 'rb')
encodings_file = pickle.load(f) 
f.close()

### Recognize a face in a video

In [5]:
vs = VideoStream(src=0).start()
time.sleep(2.0)

In [6]:
while True: 
    frame = vs.read()
    frame_fail = 0
    
    image_rgb = cv2.cvtColor(frame , cv2.COLOR_BGR2RGB)
    image_rgb = imutils.resize(frame, height = frame.shape[0], width = frame.shape[1]) 
    
    
    dets = detector(image_rgb) 
    detected_faces = []
    
    for face in dets:
        detected_faces.append(rect_to_list(face))
    
        
    encodings = fr.face_encodings(image_rgb, detected_faces) 
    
    
    #For eyes:
    for i, rect in enumerate(dets): 
        shape = predictor(image_rgb, rect) 
        shape = face_utils.shape_to_np(shape)
        
        left_eye = shape[lstart:lend] 
        right_eye = shape[rstart:rend] 
        left_EAR = eye_aspect_ratio(left_eye) 
        right_EAR = eye_aspect_ratio(right_eye)
        
        EAR = (left_EAR +right_EAR) / 2 
        
        #Draw a convex hull on top of the eyes 
        leftconvex = cv2.convexHull(left_eye)
        rightconvex = cv2.convexHull(right_eye)
        cv2.drawContours(frame, [leftconvex], -1, (0,255,0), 1)
        cv2.drawContours(frame, [rightconvex], -1 , (0,255,0), 1)
        
        if EAR < EAR_thresh:
            count += 1            
            
        else:
            if count >= BLINK_CONSEC_FRAMES:  
                blinks += 1
            
            count = 0 
        
        cv2.putText(frame, "Blinks: {}".format(blinks), (10,30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
        
    
    
    #Initialize name list to add each face
    names = []
    for encoding in encodings: 
        matches = fr.compare_faces(encodings_file['encodings'], encoding, tolerance = 0.4) 
        name  = ""
    
        if True in matches:
            match_index = [i for (i,b) in enumerate(matches) if b] 
            counts = [] 
            
            
            for i in match_index:
                counts.append(encodings_file['names'][i]) 
                
            name = Counter(counts).most_common(1)[0][0] 
        names.append(name)
    
            
    
    for ((top,right,bottom, left), name) in zip(detected_faces,names):
        cv2.rectangle(frame, (left,top), (right,bottom), (0,255,0), 2)
        cv2.putText(frame, name, (left+15,top-6), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,255,0), 2)
    
    
    #Blink Threshold to grant access:
    if blinks >= 5:
        cv2.putText(frame, "Access Granted", (60,600), cv2.FONT_HERSHEY_SIMPLEX, 5, (0,0,0), 10)
        cv2.imshow("Image", frame)
        cv2.waitKey(1)
        time.sleep(2)
        break
            
    
        
    
    cv2.imshow("Image", frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
 

cv2.destroyAllWindows()
cv2.waitKey(1)
vs.stop()




# THE END
